# Applied Data Science Capstone

This notebook will be used for the capstone project for A.

# Assignment 3: Exploring Toronto Neighborhood
<h2>Table of content</h2>
<h4>Please click on the respective submission link you would like to see</h4>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="#Submission-1">Submission 1: Building the Neighborhood data frame</a>        
    <li><a href="#Submission-2">Submission 2: Adding the geolocation</a>        
    <li><a href="#Submission-3">Submission 3: Exploring Toronto Neighborhoods</a>
        <ul>
            <li><a href="#Foursquare-exploration">Foursquare exploration</a>
            <li><a href="#Neighborhood-Analysis">Neighborhood Analysis</a>   
            <li><a href="#Cluster-Neighborhood">Cluster Neighborhood</a>
            <li><a href="Cluster-Examination">Cluster Examination</a>
        </ul>
</ul>
</div>

In [1]:
import pandas as pd
import numpy as np
import wget
import requests
import csv
import geocoder
import geopy

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Submission 1
In this section, I will be scraping the Wikipedia page to build a pandas dataframe to store the PostaCode, Borough and Neighborhood

In [3]:
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit

I downloaded the webpage as a html page to my local repository

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
print("Wiki Page Downloaded")

Wiki Page Downloaded


Now let's use the BeautifulSoup package to work with the html data

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')
print("BeautifulSoup object created")

BeautifulSoup object created


Let's get the table and view the html tag where our data is stored

In [6]:
neigh_data = soup.find_all('table', class_='wikitable sortable')

Let's save the contents of the table into a csv

In [7]:
output_rows = []
for row in soup.find_all('table', class_='wikitable sortable'):
    columns = row.find_all('tr')
    output_row = []
    for column in columns: 
        output_row.append(column.text)
    output_rows.append(output_row)
    
with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)
  

For the sake of simplicity, I went ahead and cleaned the rows in notepad++ so I can easily read it

In [8]:
#let's open the csv file
csvfile = "C:/Users/nirin/Downloads/output.csv"
df = pd.read_csv(csvfile,
                 header=0,
                 delimiter='  ',                 
                 quotechar='"',
                 error_bad_lines=False,
                 engine='python')

Skipping line 113: Expected 3 fields in line 113, saw 4. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.


In [9]:
df.shape

(179, 3)

Let's combine the neighborhood with the same Postal Code into one

In [10]:
dfPC = df.groupby(by="Postal Code", axis=0, as_index=False, sort=True, squeeze = False)
dfPC

<ipython-input-10-31838aa6381b>:1: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  dfPC = df.groupby(by="Postal Code", axis=0, as_index=False, sort=True, squeeze = False)


In [11]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Shape before removing rows

In [12]:
df.shape

(179, 3)

In [13]:
df.drop(df[df['Borough'] =='Not assigned'].index, inplace = True)
print("removed rows with Not assigned Borough")

removed rows with Not assigned Borough


Shape of the data after removing 

In [14]:
df.rename(columns={"Postal Code" :"PostalCode"})

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
159,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
164,M4Y,Downtown Toronto,Church and Wellesley
167,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
168,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
df.shape

(102, 3)

#### *** end of submission 1 ***

---------------------------------------------------------------------

## Submission 2

In this section, we will be adding two additional columns for latitude and longitude of each Borough

Let's get the csv file that contains our geospatial data and save it in a dataframe

In [16]:
geodf = pd.read_csv("C:/Users/nirin/Geospatial_Coordinates.csv",                   
                 header=0,
                 delimiter=",",                 
                 error_bad_lines=False,
                 engine='python'
                   )
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combining the two data frame, our final dataframe will look like the one below:

In [17]:
dfall = df.merge(geodf)
dfall.tail(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
97,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
98,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
99,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
100,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
101,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


In [18]:
dfall.shape

(102, 5)

### ***End of submission 2 ***

---------------------------------------------------------------------------------------

## Submission 3 Exploring Toronto

In [19]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import json 
print('Libraries imported')

Libraries imported


In [20]:
postCode = dfall['Postal Code']
torontoBorough = dfall['Borough']
torontoNeigh = dfall['Neighbourhood']
latitude = dfall['Latitude'][0]
longitude = dfall['Longitude'][1]


In [21]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfall['Borough'].unique()),
        dfall.shape[0]
    )
)

The dataframe has 10 boroughs and 102 neighborhoods.


In [22]:
dfall.dtypes

Postal Code       object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

Let's print a map of the different neighborhood of Toronto

In [23]:
map_toronto = folium.Map(location=[latitude, longitude], 
                         zoom_start = 11)

for lat, lng, label in zip(dfall['Latitude'], dfall['Longitude'], dfall['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Let's segment and cluster neighborhoods and work only with boroughs with the word "Toronto"

In [24]:
tor_data = dfall[dfall['Borough'].str.contains('Toronto')].reset_index(drop=True)
tor_data.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [25]:
postCode = tor_data['Postal Code']
borough = tor_data['Borough']
neighbor = tor_data['Neighbourhood']
latitude = tor_data['Latitude'][0]
longitude = tor_data['Longitude'][1]

In [26]:
map_tor_neigh = folium.Map(location=[latitude,longitude], zoom_start = 11)

for lat, lng, label in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor_neigh)
    
map_tor_neigh


### Foursquare exploration

In [31]:
CLIENT_ID = 'NGBR4FWG0IAT0DHXIBKYXI2E1JI2LEJMEL3L1YA1RW4CSLXG'
CLIENT_SECRET = '54NBWJITSUPUQFC5ADWUA3WL23RDMBOJTQWO53VA3D2F0JMY' 
VERSION = '20180605' 
LIMIT = 500

In this section, I will be creating a process to run through all the neighborhood list in the tor_data dataframe and pulling a list of all the vanues in the area within a 500m radius

In [32]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NGBR4FWG0IAT0DHXIBKYXI2E1JI2LEJMEL3L1YA1RW4CSLXG&client_secret=54NBWJITSUPUQFC5ADWUA3WL23RDMBOJTQWO53VA3D2F0JMY&ll=43.6542599,-79.3894938&v=20180605&limit=500'

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
tor_neigh_venues = getNearbyVenues(names = tor_data['Neighbourhood'],
                                  latitudes = tor_data['Latitude'],
                                   longitudes = tor_data['Longitude']                                    
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James T

In [36]:
tor_neigh_venues.shape

(1606, 7)

In [37]:
tor_neigh_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [38]:
tor_neigh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


Here are how many unique categories can be explored from all the venues

In [39]:
print('There are {} uniques categories.'.format(len(tor_neigh_venues['Venue Category'].unique())))

There are 237 uniques categories.


### Neighborhood Analysis

In [40]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_neigh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_neigh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.tail()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
1601,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1605,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
tor_onehot.shape

(1606, 237)

Let's group rows by neighborhood and by taking the mean of the frequency of occurence of each category

In [42]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.014706,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.014706,0.000000,0.000000,0.014706,0.00
5,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.026667,0.013333,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013333,...,0.013333,0.013333,0.000000,0.000000,0.00,0.000000,0.000000,0.013333,0.000000,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.040000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.00
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,...,0.000000,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


In [43]:
tor_grouped.shape

(38, 237)

### Neighborhood with top 5 most common venues

In [44]:
#First let's find the 5 top venues
num_topvenues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_topvenues))
    print('\n')

----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1     Beer Bar  0.04
2       Bakery  0.04
3   Restaurant  0.04
4  Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2     Coffee Shop  0.09
3  Breakfast Spot  0.09
4             Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Pizza Place  0.06
1      Recording Studio  0.06
2            Skate Park  0.06
3  Fast Food Restaurant  0.06
4                  Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2  Rental Car Location  0.06
3      Harbor / Marina  0.06
4     Sculpture Garden  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1   

In [45]:
#Let's sort the venues in descending order
def return_mostcommonvenues(row, num_topvenues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_topvenues]

Let's create a new dataframe and display the top 5 venues for each neighborhood

In [46]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_topvenues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_mostcommonvenues(tor_grouped.iloc[ind, :], num_topvenues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Restaurant,Cheese Shop,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Grocery Store
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Skate Park,Brewery,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Sculpture Garden,Boutique,Plane
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint


### Cluster Neighborhood

Let's run k-means to cluster the neighborhood into 5 clusters

In [47]:
# set number of clusters
kclusters = 5

In [48]:
tor_grouped_cluster = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster and the top 5 venues for each neighborhood

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
tor_merged = tor_data

In [50]:
# merge tor_grouped with tor_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
tor_merged.tail() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
33,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Italian Restaurant,Pub,Hotel,Seafood Restaurant
34,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0,Coffee Shop,Restaurant,Café,Pizza Place,Chinese Restaurant
35,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Hotel,American Restaurant,Gym
36,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant
37,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0,Park,Pizza Place,Skate Park,Brewery,Burrito Place


Now let's visualize the clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster Examination

#### Cluster 1

In [53]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Café
1,Downtown Toronto,0,Coffee Shop,Yoga Studio,Café,Smoothie Shop,Beer Bar
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop
3,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub
4,East Toronto,0,Pub,Trail,Health Food Store,Women's Store,Distribution Center
5,Downtown Toronto,0,Coffee Shop,Bakery,Restaurant,Cheese Shop,Beer Bar
6,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint
7,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Nightclub
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Bar
9,West Toronto,0,Pharmacy,Bakery,Grocery Store,Supermarket,Middle Eastern Restaurant


#### Cluster 2

In [54]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,Central Toronto,1,Trail,Playground,Women's Store,Deli / Bodega,Electronics Store


#### Cluster 3

In [55]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,Central Toronto,2,Park,Swim School,Bus Line,Electronics Store,Eastern European Restaurant


#### Cluster 4

In [56]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
21,Central Toronto,3,Park,Sushi Restaurant,Jewelry Store,Trail,Electronics Store
32,Downtown Toronto,3,Park,Trail,Playground,Dance Studio,Eastern European Restaurant


#### Cluster 5

In [58]:
tor_merged.loc[tor_merged['Cluster Labels'] == 5, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


## Thank you for reviewing my work :)